#  作業

Hint: 人物原始邊框座標 (60, 40), (420, 510)

請根據 Lena 圖做以下處理

- 對明亮度做直方圖均衡處理
- 水平鏡像 + 縮放處理 (0.5 倍)
- 畫出人物矩形邊框

In [55]:
import cv2
import numpy as np

img = cv2.imread('lena.png')

## Hint: 矩形

In [56]:
img_rectangle = img.copy()
cv2.rectangle(img_rectangle, (60, 40), (420, 510), (0, 0, 255), 3)

while True:
    cv2.imshow('image', img_rectangle)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## Hint: 線

In [82]:
img_line = img.copy()
cv2.line(img_line, (60, 40), (420, 510), (0, 0, 255), 3)

while True:
    cv2.imshow('image', img_line)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## Hint: 文字

In [81]:
img_text = img.copy()
cv2.putText(img_text, '(60, 40)', (60, 40), 1, 1, (0, 0, 255), 2)

while True:
    cv2.imshow('image', img_text)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 解法一
根據題目，參考之前的投影片，最簡單的方式為
1. 顏色的操作（對明亮度做直方圖均衡）
2. 把舉行邊框畫在圖片上之後在做位置上的變換
得到的圖示正確的結果，但你不知道新的矩形邊框座標點

In [59]:
img_hw = img.copy()
point1 = (60, 40)
point2 = (420, 510)

# 直方圖均衡
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

# 畫出矩形邊框
cv2.rectangle(img_hw, point1, point2, (0, 0, 255), 3)

# 水平鏡像 + 縮放處理（0.5 倍）
img_hw = img_hw[:, ::-1, :]

img_hw = cv2.resize(img_hw, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

while True:
    cv2.imshow("image", img_hw)
    k = cv2.waitKey(0)
    if k==27:
        cv2.destroyAllWindows()
        break

##  解法二

如果希望得知矩型邊框的位置，可以：

1. 顏色的操作 (對明亮度做直方圖均衡)
2. 鏡像尺寸可以透過四則運算得知
3. 建構 transformation matrix，並縮放圖片
4. 把矩型邊框的點與 transformation matrix 相乘就會得到縮放後的位置
5. 畫圖

In [60]:
img_hw = img.copy()
point1 = [60, 40]
point2 = [420, 510]

# 做直方圖均衡
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

# 水平鏡像
img_hw = img_hw[:, ::-1, :]

# 透過四則運算計算鏡像後的位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
w = img_hw.shape[0]
point1[0] = w - point1[0]
point2[0] = w - point2[0]
point1[0], point2[0] = point2[0], point1[0]

"""
縮放處理 (0.5 倍)
"""
fx = 0.5
fy = 0.5
resize_col = int(img_hw.shape[1] * fx)
resize_row = int(img_hw.shape[0] * fy)

# 建構 scale matrix
matrix_scale = np.array([[fx, 0, 0], [0, fy, 0]], dtype=np.float32)
img_hw = cv2.warpAffine(img_hw, matrix_scale, (resize_col, resize_row))

# 把左上跟右下轉為矩陣型式
original_point = np.array((point1, point2), dtype=np.float32)
# print('M_scale.shape={}, bbox.shape={}'.format(matrix_scale.shape, bbox.shape))

# 做矩陣乘法可以使用 `np.dot`, 而為了做矩陣乘法, 因此 matrix_scale 需要做轉置之後才能相乘
# matrix.shpae = [2, 3]
# original_point.shpae = [2, 2]
resize_point = np.dot(matrix_scale.T, original_point).astype('uint8')

# 畫圖的點所需要的型態為 tuple
resize_point1 = tuple(resize_point[0])
resize_point2 = tuple(resize_point[1])

print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(resize_point1, resize_point2))

# 畫圖
cv2.rectangle(img_hw, resize_point1, resize_point2, (0, 0, 255), 3)

while True:
    cv2.imshow('image', img_hw)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

origin point1=[92, 40], origin point2=[452, 510]
resize point1=(46, 20), resize point2=(226, 255)


##  解法三 (optional)

把矩型邊框用遮罩的方式呈現，使用同樣處理圖片的方式處理遮罩
最後再找遮罩的左上跟右下的點的位置

這邊會用到許多沒提過的東西，所以當作 optional

而下面兩個分別是使用 雙線性差值 與 transformation matrix，我們可以觀察到其有些微的不同

In [1]:
# 尋找矩形的 size

img_hw = img.copy()
point1 = (60, 40)
point2 = (420, 510)

# 2D mask
# 取得與 image 一樣大小的零矩陣
img_mask = np.zeros_like(img_hw)
img_mask[40:510, 60:420, :] = 255

# 做直方圖均衡
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

# 水平鏡像
img_hw = img_hw[:, ::-1, :]
img_mask = img_mask[:, ::-1, :]

# 縮放處理
img_hw = cv2.resize(img_hw, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
img_mask = cv2.resize(img_hw, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

# 取得遮罩的左上和右下的點
# 取得 img_mask 非零的座標
height_coordinate, width_coordinate, channel_coordinate = np.where(img_mask)

resize_point1 = (min(width_coordinate), min(height_coordinate))
resize_point2 = (max(width_coordinate), max(height_coordinate))

# 印出原有和更改後的左上、右下兩個點
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(scale_point1, scale_point2))

# 畫出矩形邊框
cv2.rectangle(img_hw, resize_point1, resize_point2, (0, 0, 255), 3)

while True:
    cv2.imshow("image", img_hw)
    k = cv2.waitKey(0)

    if k ==27 :
        cv2.destroyAllWindows()
        break


NameError: name 'img' is not defined

In [84]:
img_hw = img.copy()
point1 = [60, 40]
point2 = [420, 510]

# 做直方圖均衡
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_BGR2HSV)
img_hw[..., -1] = cv2.equalizeHist(img_hw[..., -1])
img_hw = cv2.cvtColor(img_hw, cv2.COLOR_HSV2BGR)

# 水平鏡像
img_hw = img_hw[:, ::-1, :]

# 透過四則運算計算鏡像後位置
# 確保點的位置一樣是左上跟右下，所以交換鏡像後的 x 座標 (y 座標做水平鏡像後位置不變)
w = img_hw.shape[1]
point1[0] = w - point1[0]
point2[0] = w - point2[0]
point1[0], point2[0] = point2[0], point1[0]

# 縮放處理（0.5 倍）
fx = 0.5
fy = 0.5
resize_col = int(img_hw.shape[1] * fx)
resize_row = int(img_hw.shape[0] * fy)

# 建構 scale matrix
matrix_scale = np.array([[fx, 0, 0],
                    [0, fy, 0]], dtype=np.float32)
img_hw = cv2.warpAffine(img_hw, matrix_scale, (resize_col, resize_row))

# 把左上跟右下轉為矩陣型式
original_point = np.array((point1, point2), dtype=np.float32)
#print('matrix_scale.shape={}, original.shape={}'.format(matrix_scale.shape, original.shape))

# 做矩陣乘法可以使用 `np.dot`, 為了做矩陣乘法, matrix_scale 需要做轉置之後才能相乘
homo_coor_result = np.dot(matrix_scale.T, original_point)
homo_coor_result = homo_coor_result.astype('uint8')

resize_point1 = tuple(homo_coor_result[0])
resize_point2 = tuple(homo_coor_result[1])

# 印出原有和更改後的左上、右下兩個點
print('origin point1={}, origin point2={}'.format(point1, point2))
print('resize point1={}, resize point2={}'.format(resize_point1, resize_point2))

# 畫圖遮罩
# color=255 is meant (255, 255, 255)
rectangle_array = np.zeros(img_hw.shape[:2], dtype = "uint8")
mask = cv2.rectangle(rectangle_array, resize_point1, resize_point2, 255, -1)

# Apply out mask
# 將圖片與 mask 做 and 運算 
masked = cv2.bitwise_and(img_hw, img_hw, mask=mask)

while True:
    # cv2.imshow('image', img)
    # cv2.imshow('image_hw', img_hw)
    cv2.imshow('Masked', masked)
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break